In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [76]:
data = pd.read_csv('train.csv')

In [77]:
data

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [78]:
data = np.array(data)
m,n = data.shape
np.random.shuffle(data)


In [79]:
data_dev = data[0:1000].T
y_dev = data_dev[0]
X_dev = data_dev[1:n]

data_train = data[1000:m].T
y_train = data_train[0]
X_train = data_train[1:n]

X_train = X_train / 255
X_dev = X_dev / 255



In [80]:
X_train[:].shape

(784, 41000)

In [81]:
def init_params():
    W1 = np.random.randn(10, 784) * np.sqrt(1 / 784)  # Xavier Initialization
    b1 = np.zeros((10, 1))
    W2 = np.random.randn(10, 10) * np.sqrt(1 / 10)
    b2 = np.zeros((10, 1))
    return W1, b1, W2, b2


def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    shift_Z = Z - np.max(Z, axis=0, keepdims=True)  # Subtract max for stability
    exp_Z = np.exp(shift_Z)
    return exp_Z / np.sum(exp_Z, axis=0, keepdims=True)


def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return (Z > 0).astype(float)


def one_hot(Y, num_classes=10):
    one_hot_Y = np.zeros((num_classes, Y.size))
    one_hot_Y[Y, np.arange(Y.size)] = 1
    return one_hot_Y


def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    m = X.shape[1]
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1, axis=1, keepdims=True)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 -= alpha * dW1
    b1 -= alpha * db1
    W2 -= alpha * dW2
    b2 -= alpha * db2
    return W1, b1, W2, b2





In [82]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        
        if i % 10 == 0:
            predictions = get_predictions(A2)
            acc = get_accuracy(predictions, Y)
            print(f"Iteration {i}: Accuracy = {acc:.4f}")

    return W1, b1, W2, b2


In [ ]:
W1, b1, W2, b2 = gradient_descent(X_train, y_train, 0.10, 500)

[0 5 0 ... 0 0 5] [2 7 4 ... 7 6 5]
Iteration 0: Accuracy = 0.1279
[0 0 0 ... 0 2 0] [2 7 4 ... 7 6 5]
Iteration 10: Accuracy = 0.3229
[2 0 0 ... 0 2 0] [2 7 4 ... 7 6 5]
Iteration 20: Accuracy = 0.5473
[2 9 0 ... 7 6 0] [2 7 4 ... 7 6 5]
Iteration 30: Accuracy = 0.6615
[2 7 7 ... 7 6 0] [2 7 4 ... 7 6 5]
Iteration 40: Accuracy = 0.7337
[2 7 7 ... 7 6 0] [2 7 4 ... 7 6 5]
Iteration 50: Accuracy = 0.7646
[2 7 6 ... 7 6 0] [2 7 4 ... 7 6 5]
Iteration 60: Accuracy = 0.7872
[2 7 6 ... 7 6 5] [2 7 4 ... 7 6 5]
Iteration 70: Accuracy = 0.8047
[2 7 6 ... 7 6 5] [2 7 4 ... 7 6 5]
Iteration 80: Accuracy = 0.8171
[2 7 6 ... 7 6 5] [2 7 4 ... 7 6 5]
Iteration 90: Accuracy = 0.8276
[2 7 6 ... 7 6 5] [2 7 4 ... 7 6 5]
Iteration 100: Accuracy = 0.8352
[2 7 6 ... 7 6 5] [2 7 4 ... 7 6 5]
Iteration 110: Accuracy = 0.8421
[2 7 6 ... 7 6 5] [2 7 4 ... 7 6 5]
Iteration 120: Accuracy = 0.8474
[2 7 6 ... 7 6 5] [2 7 4 ... 7 6 5]
Iteration 130: Accuracy = 0.8520
[2 7 6 ... 7 6 5] [2 7 4 ... 7 6 5]
Iteration

In [ ]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(index, W1, b1, W2, b2):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2)
    label = y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)
    
    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

In [ ]:
test_prediction(180, W1, b1, W2, b2)
test_prediction(130, W1, b1, W2, b2)
test_prediction(202, W1, b1, W2, b2)
test_prediction(306, W1, b1, W2, b2)

In [ ]:
dev_predictions = make_predictions(X_dev, W1, b1, W2, b2)
get_accuracy(dev_predictions, y_dev)